In [3]:
!pip install tensorflow numpy pandas scikit-learn opencv-python-headless

In [4]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from sklearn.neighbors import NearestNeighbors
import pandas as pd


In [5]:
model = MobileNetV2(weights='imagenet')


14536120/14536120 [==============================] - 0s 0us/step


In [6]:
def recognize_image(image_path):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (224, 224))
    image_array = np.expand_dims(image_resized, axis=0)
    image_preprocessed = preprocess_input(image_array)

    predictions = model.predict(image_preprocessed)
    decoded_predictions = decode_predictions(predictions, top=5)[0]

    return decoded_predictions


In [13]:

product_db = pd.DataFrame({
    'product_id': [1, 2, 3, 4, 5],
    'product_name': ['Shirt', 'Jeans', 'Shoes', 'Hat', 'Jacket'],
    'features': [
        np.array([0.1, 0.2, 0.3, 0.4]),  # Example feature vectors
        np.array([0.2, 0.1, 0.4, 0.3]),
        np.array([0.3, 0.4, 0.1, 0.2]),
        np.array([0.4, 0.3, 0.2, 0.1]),
        np.array([0.5, 0.5, 0.5, 0.5])
    ]
})


def search_product(query_features, k=5):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(list(product_db['features']))
    distances, indices = nbrs.kneighbors([query_features])

    return product_db.iloc[indices[0]]


In [14]:
# Example user purchase history
user_history = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2],
    'product_id': [1, 2, 3, 4, 5]
})

def recommend_products(user_id, n_recommendations=3):
    user_purchases = user_history[user_history['user_id'] == user_id]['product_id']
    user_features = product_db[product_db['product_id'].isin(user_purchases)]['features'].mean()

    nbrs = NearestNeighbors(n_neighbors=n_recommendations, algorithm='ball_tree').fit(list(product_db['features']))
    distances, indices = nbrs.kneighbors([user_features])

    return product_db.iloc[indices[0]]


In [15]:
def identify_items_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            image_resized = cv2.resize(frame, (224, 224))
            image_array = np.expand_dims(image_resized, axis=0)
            image_preprocessed = preprocess_input(image_array)

            predictions = model.predict(image_preprocessed)
            decoded_predictions = decode_predictions(predictions, top=5)[0]

            print("Frame Predictions:", decoded_predictions)
        else:
            break

    cap.release()


In [18]:

# Example for Image Recognition
image_path = "/content/drive/MyDrive/Amazon Hackathon/amazon-2.jpeg"  # Adjust path to your uploaded image
predictions = recognize_image(image_path)
print("Predictions:", predictions)

# Example for Product Search
query_features = np.array([0.15, 0.25, 0.35, 0.45])  # Example feature vector from image recognition
search_results = search_product(query_features)
print("Search Results:")
print(search_results)

# Example for Personalized Recommendations
user_id = 1
recommendations = recommend_products(user_id)
print("Recommendations for User", user_id, ":")
print(recommendations)

# Example for Item Identification in Video
# video_path = "/content/drive/MyDrive/Amazon Hackathon/6615054-uhd_3840_2160_25fps.mp4"  # Adjust path to your uploaded video
# identify_items_in_video(video_path)


1/1 [==============================] - 0s 62ms/step
Predictions: [('n04355933', 'sunglass', 0.048297856), ('n03124043', 'cowboy_boot', 0.03995881), ('n02963159', 'cardigan', 0.03707328), ('n03000684', 'chain_saw', 0.0346261), ('n03709823', 'mailbag', 0.03314963)]
Search Results:
   product_id product_name              features
0           1        Shirt  [0.1, 0.2, 0.3, 0.4]
1           2        Jeans  [0.2, 0.1, 0.4, 0.3]
2           3        Shoes  [0.3, 0.4, 0.1, 0.2]
3           4          Hat  [0.4, 0.3, 0.2, 0.1]
4           5       Jacket  [0.5, 0.5, 0.5, 0.5]
Recommendations for User 1 :
   product_id product_name              features
0           1        Shirt  [0.1, 0.2, 0.3, 0.4]
1           2        Jeans  [0.2, 0.1, 0.4, 0.3]
2           3        Shoes  [0.3, 0.4, 0.1, 0.2]
